In [1]:
## use tf.data.TextLineDataset to load examples from text files.

import tensorflow as tf

import tensorflow_datasets as tfds
import os

In [3]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

'C:\\Users\\Nano\\.keras\\datasets'

In [4]:
# Load text into datasets

def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [5]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [6]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [7]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b"And, dying, of the dead relax'd his grasp.">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Of Hector, him bore safe with all his arms'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'horses here and make the reins fast to the rim of the chariot; then be'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Our ships, at least a prize not lightly won.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"And slay thee, not upon the fun'ral bed,">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


In [8]:
# Encode text lines as numbers
# Build vocabulary

tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

In [10]:
# Encode examples

encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [13]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b"And, dying, of the dead relax'd his grasp."


In [14]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[15454, 15135, 12958, 15610, 11244, 16232, 16243, 5780, 16048]


In [15]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

In [18]:
# Dataset.map runs in graph mode.
# wrap it in a tf.py_function. 
# The tf.py_function will pass regular tensors (with a value and a .numpy() method to access it), to the wrapped python function.

def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

In [19]:
# Split the dataset into test and train batches
# Use tf.data.Dataset.take and tf.data.Dataset.skip to create a small test dataset and a larger training set.

train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [20]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]




(<tf.Tensor: shape=(15,), dtype=int64, numpy=
 array([15454, 15135, 12958, 15610, 11244, 16232, 16243,  5780, 16048,
            0,     0,     0,     0,     0,     0], dtype=int64)>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

In [21]:
vocab_size += 1

In [22]:
# Build the model

model = tf.keras.Sequential()

In [23]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [24]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [25]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))

In [26]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [27]:
# Train the model

model.fit(train_data, epochs=3, validation_data=test_data)

Instructions for updating:
Use `tf.data.Dataset.cardinality()
Epoch 1/3
697/697 [==============================] - 19s 28ms/step - loss: 0.5222 - accuracy: 0.7451 - val_loss: 0.3778 - val_accuracy: 0.8256
Epoch 2/3
697/697 [==============================] - 20s 29ms/step - loss: 0.2996 - accuracy: 0.8672 - val_loss: 0.3781 - val_accuracy: 0.8224
Epoch 3/3
697/697 [==============================] - 20s 29ms/step - loss: 0.2268 - accuracy: 0.9022 - val_loss: 0.3825 - val_accuracy: 0.8354


In [28]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

79/79 [==============================] - 2s 22ms/step - loss: 0.3825 - accuracy: 0.8354

Eval loss: 0.382, Eval accuracy: 0.835
